In [2]:
import pycram
from pycram.bullet_world import BulletWorld, Object
import pycram.bullet_world_reasoning as btr
import tf
from pycram.designators.motion_designator import MotionDesignatorDescription, MoveArmJointsMotion
from pycram.process_module import simulated_robot, with_simulated_robot
from pycram.language import macros, par
from pycram.designators.location_designator import *
from pycram.designators.action_designator import *
from pycram.enums import Arms
from pycram.designators.object_designator import *
from pycram.designators.object_designator import BelieveObject

world = BulletWorld()
world.set_gravity([0, 0, -9.8])
#plane = Object("floor", "environment", "plane.urdf", world=world)
robot = Object("pr2", "robot", "../../resources/" + robot_description.name + ".urdf")
robot_desig = ObjectDesignatorDescription(names=["pr2"]).resolve()


kitchen = Object("kitchen", "environment", "kitchen.urdf")



robot.set_joint_state(robot_description.torso_joint, 0.24)
kitchen_desig = ObjectDesignatorDescription(names=["kitchen"])

kitchen.set_color([0.2, 0, 0.4, 0.6])


Unknown attribute "type" in /robot[@name='pr2']/link[@name='base_laser_link']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='wide_stereo_optical_frame']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='narrow_stereo_optical_frame']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='laser_tilt_link']
Exception in thread Thread-10:
Traceback (most recent call last):
  File "/usr/lib/python3.8/threading.py", line 932, in _bootstrap_inner
    self.run()
  File "/home/oem/pycram_ws/src/pycram/src/pycram/bullet_world.py", line 536, in run
    basePosition=cameraTargetPosition)
UnboundLocalError: local variable 'cameraTargetPosition' referenced before assignment


startThreads creating 1 threads.
starting thread 0
started thread 0 
argc=2
argv[0] = --unused
argv[1] = --start_demo_name=Physics Server
ExampleBrowserThreadFunc started
X11 functions dynamically loaded using dlopen/dlsym OK!
X11 functions dynamically loaded using dlopen/dlsym OK!
Creating context
Created GL 3.3 context
Direct GLX rendering context obtained
Making context current
GL_VENDOR=NVIDIA Corporation
GL_RENDERER=NVIDIA GeForce RTX 2060/PCIe/SSE2
GL_VERSION=3.3.0 NVIDIA 525.125.06
GL_SHADING_LANGUAGE_VERSION=3.30 NVIDIA via Cg compiler
pthread_getconcurrency()=0
Version = 3.3.0 NVIDIA 525.125.06
Vendor = NVIDIA Corporation
Renderer = NVIDIA GeForce RTX 2060/PCIe/SSE2
b3Printf: Selected demo: Physics Server
startThreads creating 1 threads.
starting thread 0
started thread 0 
MotionThreadFunc thread started


Unknown tag "material" in /robot[@name='plane']/link[@name='planeLink']/collision[1]
Unknown tag "contact" in /robot[@name='plane']/link[@name='planeLink']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='base_laser_link']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='wide_stereo_optical_frame']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='narrow_stereo_optical_frame']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='laser_tilt_link']
Unknown tag "material" in /robot[@name='plane']/link[@name='planeLink']/collision[1]
Unknown tag "contact" in /robot[@name='plane']/link[@name='planeLink']
Scalar element defined multiple times: limit
Unknown attribute "type" in /robot[@name='pr2']/link[@name='base_laser_link']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='wide_stereo_optical_frame']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='narrow_stereo_optical_frame']
Unknown attribute "type" in /robot[@name='pr2']/link[

In [3]:
spawning_poses = {
    #'bigknife': Pose([-0.95, 1.2, 1.3], [1, -1, 1, -1]),
    'bigknife': Pose([-0.35, 1.2, 1.2], [0, 0, 0, 1]),
    'bread': Pose([-0.85, 0.9, 0.95],[0, 0, -1, 1])
}
bigknife = Object("bigknife", "bigkinfe", "big-knife.stl", spawning_poses["bigknife"])
bread = Object("bread", "bread", "bread.stl", spawning_poses["bread"])
bigknife_BO = BelieveObject(names=["bigknife"])
bread_BO = BelieveObject(names=["bread"])


In [4]:
with simulated_robot:   
    ParkArmsAction([Arms.BOTH]).resolve().perform()

    MoveTorsoAction([0.33]).resolve().perform()

    pickup_pose_knife = CostmapLocation(target=bigknife_BO.resolve(), reachable_for=robot_desig).resolve()
    pickup_arm = pickup_pose_knife.reachable_arms[0]

    NavigateAction(target_locations=[pickup_pose_knife.pose]).resolve().perform()
    #PickUpAction(object_designator_description=bigknife_BO, arms=[left], grasps=["front"]).resolve().perform()
    PickUpAction(object_designator_description=bigknife_BO, 
                     arms=["left"], 
                     grasps=["top"]).resolve().perform()
    ParkArmsAction([Arms.BOTH]).resolve().perform()

In [5]:
with simulated_robot:   
    ParkArmsAction([Arms.BOTH]).resolve().perform()

    MoveTorsoAction([0.33]).resolve().perform()
    
    cutting_standing_pose = CostmapLocation(target=bread_BO.resolve(), reachable_for=robot_desig).resolve()
    NavigateAction(target_locations=[cutting_standing_pose.pose]).resolve().perform()
 # MoveTCPMotion(target=self.pouring_location, arm=self.arm).resolve(). \
 #                perform()
 #            # sleep for some seconds
 #            time.sleep(self.wait_duration) # Sleep for wait_duration seconds
 #            MoveTCPMotion(target=self.revert_location, arm=self.arm).resolve(). \
 #                perform()
    LookAtAction(targets=[bread_BO.resolve().pose]).resolve().perform()
    
    detected_bread_desig = DetectAction(bread_BO).resolve().perform()
    
    
    


In [6]:
import numpy as np

def calculate_and_transform_slices(bread_dimension, slice_thickness, bread_pose):
    """
    Calculate number of slices, transform them to the map frame, and provide coordinates with position and orientation.

    Args:
    - bread_dimension (tuple): Dimensions of the bread (length, width, height)
    - slice_thickness (float): Desired thickness of each slice in meters
    - bread_pose (Pose): Spawning pose of the bread with position and orientation

    Returns:
    - num_slices (int): Number of slices
    - coordinates_with_orientation (list): List of slice coordinates with position and orientation
    """
    
    # Calculate number of slices
    bread_width = bread_dimension[1]
    num_slices = int(bread_width // slice_thickness)
    
    # Calculate the starting Y-coordinate offset (half the width minus half a slice thickness)
    start_offset = -bread_width / 2 + slice_thickness / 2
    
    # Calculate slice coordinates
    slice_coordinates = [start_offset + i * slice_thickness for i in range(num_slices)]
    
    # Transform slice coordinates to map frame with orientation
    coordinates_with_orientation = []
    for coordinate in slice_coordinates:
        x_in_map = bread_pose.position.x 
        y_in_map = bread_pose.position.y + coordinate
        z_in_map = bread_pose.position.z
        
        orientation = bread_pose.orientation
        coordinates_with_orientation.append({
            "position": [x_in_map, y_in_map, z_in_map],
            "orientation": orientation
        })
        
    return num_slices, coordinates_with_orientation




# Assuming bread_dim contains the dimensions of the bread (Placeholder dimensions for now, adjust as per actual value)
#bread_dim =  bread.get_Object_Dimensions()
bread_dim = (0.6, 0.2, 0.1)
# Given slice thickness is 3 cm or 0.03 meters
slice_thickness = 0.01
# Calculate slices and transform them to the map frame with orientation
num_slices, coordinates_with_orientation = calculate_and_transform_slices(bread_dim, slice_thickness, spawning_poses['bread'])
# Calculate slices and transform them to the map frame with orientation using the updated function




print(coordinates_with_orientation)


[{'position': [-0.85, 0.805, 0.95], 'orientation': x: 0.0
y: 0.0
z: -0.7071067811865475
w: 0.7071067811865475}, {'position': [-0.85, 0.8150000000000001, 0.95], 'orientation': x: 0.0
y: 0.0
z: -0.7071067811865475
w: 0.7071067811865475}, {'position': [-0.85, 0.8250000000000001, 0.95], 'orientation': x: 0.0
y: 0.0
z: -0.7071067811865475
w: 0.7071067811865475}, {'position': [-0.85, 0.835, 0.95], 'orientation': x: 0.0
y: 0.0
z: -0.7071067811865475
w: 0.7071067811865475}, {'position': [-0.85, 0.845, 0.95], 'orientation': x: 0.0
y: 0.0
z: -0.7071067811865475
w: 0.7071067811865475}, {'position': [-0.85, 0.855, 0.95], 'orientation': x: 0.0
y: 0.0
z: -0.7071067811865475
w: 0.7071067811865475}, {'position': [-0.85, 0.865, 0.95], 'orientation': x: 0.0
y: 0.0
z: -0.7071067811865475
w: 0.7071067811865475}, {'position': [-0.85, 0.875, 0.95], 'orientation': x: 0.0
y: 0.0
z: -0.7071067811865475
w: 0.7071067811865475}, {'position': [-0.85, 0.885, 0.95], 'orientation': x: 0.0
y: 0.0
z: -0.707106781186547

In [7]:
# Assuming you have an instance of the class that contains the add_vis_axis method

for coord in coordinates_with_orientation:
    # Convert the dictionary format to Pose object
    pose = Pose(coord["position"], coord["orientation"])

    world.add_vis_axis(pose)

In [13]:
 bread_dim =  bread.get_Object_Dimensions()
print(bread_dim)

[0.10109814563393593, 0.2661544748544694, 0.06903131799399853]


In [1]:
world.remove_vis_axis()

NameError: name 'world' is not defined